### IO Class Hierarchy 
Java supports two types of streams:
- byte streams : read data in bytes
- character streams : read data as characters

Separate classes are there for reading data and writing data. For byte streams, `InputStream` and `OutputStream` are at the top of the hierarchy. For character stream `Reader` and `Writer` are at the top of the hierarchy.

### Byte Stream
Hierarchy
```
InputStream
        |
        +--- FileInputStream : obtain bytes from a file
        +--- AudioInputStream: obtain bytes from an audio stream
        +--- ObjectInputStream: for reading serialized objects
        +--- FilterInputStream: transforms other input streams
        |                   |
        |                   +--- BufferedInputStream
        +--- PipedInputStream
        +--- ...                      
```
Similarly
```
OutputStream
        |
        +--- FileOutputStream
        +--- ObjectOutputStream
        +--- FilterOutputStream
        |                   |
        |                   +--- BufferedOutputStream
        +--- PipedOutputStream
        +--- ...                      
```

### Some Use Cases
1. Reading text file using InputStream. Reading a utf-8 encoded file with foreign characters will not give correct result if characters are outside ASCII range.  

```java
public void readTextFile(String path) {
		try(InputStream in = new FileInputStream(new File(path))){
			int data = -1;
			while((data = in.read())!=-1) {
				System.out.print((char)data);	// Even though char is two bytes big
			}									// data will always be 1 byte long
		} catch (FileNotFoundException e) {
			e.printStackTrace();
		} catch (IOException e) {
			e.printStackTrace();
		}
	}
```

2. Reading header of a wav file. A wave file's header is always "RIFF".  

```java
public void readWaveHeader(String path) {
		try(InputStream in = new FileInputStream(new File(path))){
			byte[] headerBytes = new byte[4];
			in.read(headerBytes, 0, 4);
			for(int i=0;i<4;i++) {
				System.out.print((char)headerBytes[i]);
			}
		} catch (FileNotFoundException e) {
			e.printStackTrace();
		} catch (IOException e) {
			e.printStackTrace();
		}
	}
```

3. Writing to a file  

```java
public static void writeTables(String path, int start, int end) {
		try(OutputStream out = new FileOutputStream(new File(path))){
			for(int i=start;i<=end;i++) {
				for(int j=1;j<10;j++) {
					int result = i*j;
					String line = "" + i + " X " + j + " = " + result + System.lineSeparator();
					out.write(line.getBytes());
				}
			}
		} catch (FileNotFoundException e) {
			e.printStackTrace();
		} catch (IOException e) {
			e.printStackTrace();
		}
	}
```

### Buffered Byte Stream
If multiple reads or writes are to be made to a stream, it is better to use Buffered Input/Output stream. Buffered streams maintain a small byte buffer to/from which bytes are written/read. We supply a stream as constructor parameter to buffered stream.

```java
	public static void readTextFile(String path) {
		try(InputStream in = new BufferedInputStream(new FileInputStream(new File(path)))){
			int data = -1;
			while((data = in.read())!=-1) {
				System.out.print((char)data);	
			}									
		} catch (FileNotFoundException e) {
			e.printStackTrace();
		} catch (IOException e) {
			e.printStackTrace();
		}
	}
```

One additional benefit is that these classes support `mark` and `reset` operations.

### Character Streams
Hierarchy
```
Reader
    |
    +--- BufferedReader
    +--- InputStreamReader: bridge between byte and character stream. Lets you set encoding.
    |                   |
    |                   +--- FileReader
    +--- StringReader
    +--- FilterReader
    +--- ...
```
For Writer,
```
Writer
    |
    +--- BufferedWriter
    +--- OutputStreamWriter
    |                   |
    |                   +--- FileWriter
    +--- StringWriter
    +--- FilterWriter
    +--- ...
```

### Use Cases
1. Reading a text file. FileReader uses platform default encoding, which isn't always a great idea. It may fail to read foreign characters properly.  

```java
public static void readSomeCharacters(String path, int amount) {
		try(Reader r = new FileReader(path)){
			char[] data = new char[amount];
			r.read(data,0,amount);
			System.out.print(data);
		} catch (FileNotFoundException e) {
			e.printStackTrace();
		} catch (IOException e) {
			e.printStackTrace();
		}
	}
```

2. BufferedReader is the better choice in most cases. Above example wrapped in BufferedReader:  

```java
public static void readLines(String path) {
		try(BufferedReader br = new BufferedReader(new FileReader(path))){
			String line = null;
			while((line=br.readLine()) != null) {
				System.out.println(line);
			}
		} catch (FileNotFoundException e) {
			e.printStackTrace();
		} catch (IOException e) {
			e.printStackTrace();
		}
	}
```

Additionally, BufferedReader can read entire line at a time.

3. Writing UTF-16 encoded content.

```java
public void writeLines(String path) {
		try(BufferedWriter bw = new BufferedWriter(
				new OutputStreamWriter(
						new FileOutputStream(
								new File(path)), "UTF-16"))){
			String text = "تشكيل الحروف";
			bw.write(text);
		} catch (UnsupportedEncodingException e) {
			e.printStackTrace();
		} catch (FileNotFoundException e) {
			e.printStackTrace();
		} catch (IOException e) {
			e.printStackTrace();
		}
	}
```